In [1]:
import pandas as pd
import numpy as np
import os
import sklearn.metrics
from sklearn.metrics import f1_score
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target
#from joblib import Parallel, delayed
import multiprocessing
import itertools
from sklearn.ensemble import GradientBoostingClassifier as gbm 
from sklearn.cross_validation import KFold
os.getcwd()

'/Users/a0v0022/Documents/Logical_Purchase'

In [2]:
data = pd.read_csv("Logical_purchase_1000.csv")

In [3]:
data['add_to_cart_order'] = data.groupby(['user_id', 'order_id'])['product_id'].rank(method='dense').astype(int)
data['eval_set'] = ['prior' if x > 1 else 'train' for x in data['train_test']]
data.rename(columns={'order_id': 'order_number'}, inplace=True)
data['order_id']= data.apply(lambda x:'%s_%s' % (x['user_id'],x['order_number']),axis=1)

In [4]:
order_prior = data[data['eval_set'] == 'prior']

In [5]:
order_train = data[data['train_test'] == 1]

In [6]:
data.columns

Index([u'user_id', u'visit_date', u'product_id', u'units', u'dollars',
       u'cust_lead_date', u'cust_prod_lead_date', u'order_number',
       u'reordered', u'days_since_prior_order', u'cust_prod_aog', u'order_dow',
       u'week_day_name', u'cal_month_nbr', u'cal_month_name', u'cal_qtr_nbr',
       u'cal_year_nbr', u'cal_week_nbr', u'rank_cust', u'train_test',
       u'add_to_cart_order', u'eval_set', u'order_id'],
      dtype='object')

In [7]:
cust_map = data.loc[:,['user_id', 'rank_cust']]
cust_map = cust_map.drop_duplicates()
cust_map = pd.DataFrame(cust_map)

In [8]:
orders = data.loc[: , ['user_id' , 'order_id' ,'order_number', 'days_since_prior_order','eval_set', 'order_dow']]
orders = orders.drop_duplicates()
orders = pd.DataFrame(orders)

In [9]:
cust_map.shape

(1000, 2)

In [10]:
order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
['days_since_prior_order'].sum().groupby(level=[0]).cumsum().reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})
order_comsum.columns = ['user_id', 'order_number', 'days_since_prior_order_comsum']

In [11]:
order_comsum = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]

In [12]:
order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id']]

In [13]:
order_product = pd.merge(order_product, order_comsum, on='order_id')

In [14]:
temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
temp = temp.to_frame('periods').reset_index()

In [15]:
aggregated = temp.copy()

In [16]:
aggregated['last'] = aggregated.periods.apply(lambda x: x[-1] if len(x) > 0 else np.nan)
aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
aggregated.drop('periods', axis=1, inplace=True)

/Users/a0v0022/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [17]:
LP_check = aggregated[(aggregated['user_id'] == 1005126395)]
LP_check.to_csv('LP_check4.csv', index=False, header=True)

In [18]:
prob = order_prior
prob = prob.groupby(['product_id', 'user_id'],as_index=False).agg({'reordered':'sum', 'train_test': 'size'})
prob.rename(columns={'sum': 'reordered', 'train_test': 'total'}, inplace=True)

In [19]:
prob.reordered = (prob.reordered > 0).astype(np.float32)
prob.total = (prob.total > 0).astype(np.float32)
prob['reorder_prob'] = prob.reordered / prob.total
prob = prob.groupby('product_id').agg({'reorder_prob': 'mean'}).rename(columns={'mean': 'reorder_prob'}).reset_index()

In [20]:
prod_stat = order_prior.groupby('product_id').agg({'reordered': ['sum', 'size']})
prod_stat.columns = prod_stat.columns.levels[1]
prod_stat.rename(columns={'sum':'prod_reorders',
                          'size':'prod_orders'}, inplace=True)
prod_stat.reset_index(inplace=True)
prod_stat['reorder_ration'] = prod_stat['prod_reorders'] / prod_stat['prod_orders']
prod_stat = pd.merge(prod_stat, prob, on='product_id')

In [22]:
user_stat = orders.loc[orders.eval_set == 'prior', :].groupby('user_id').agg({'order_number': 'max',
                                                                              'days_since_prior_order': ['sum',
                                                                                                         'mean',
                                                                                                         'median']})
user_stat.columns = user_stat.columns.droplevel(0)
user_stat.rename(columns={'max': 'user_orders',
                          'sum': 'user_order_starts_at',
                          'mean': 'user_mean_days_since_prior',
                          'median': 'user_median_days_since_prior'}, inplace=True)
user_stat.reset_index(inplace=True)

In [24]:
orders_products =  order_prior
user_order_stat = orders_products.loc[order_prior.eval_set == 'prior', :].groupby('user_id').agg({'user_id': 'size',
                                                          'reordered': 'sum',
                                                          "product_id": lambda x: x.nunique()})
user_order_stat.rename(columns={'user_id': 'user_total_products',
                                'product_id': 'user_distinct_products',
                                'reordered': 'user_reorder_ratio'}, inplace=True)
user_order_stat.reset_index(inplace=True)
user_order_stat.user_reorder_ratio = user_order_stat.user_reorder_ratio / user_order_stat.user_total_products

In [26]:
user_stat = pd.merge(user_stat, user_order_stat, on='user_id')


In [27]:
user_stat['user_average_basket'] = user_stat.user_total_products / user_stat.user_orders

In [29]:
prod_usr = order_prior.groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
prod_usr.rename(columns={'user_id':'prod_users_unq'}, inplace=True)
prod_usr.reset_index(inplace=True)
prod_usr_reordered = order_prior.loc[order_prior.reordered, :].groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
prod_usr_reordered.rename(columns={'user_id': 'prod_users_unq_reordered'}, inplace=True)
prod_usr_reordered.reset_index(inplace=True)

In [30]:
orders_products = order_prior
order_stat = orders_products.groupby('order_id').agg({'order_id': 'size'}) \
    .rename(columns={'order_id': 'order_size'}).reset_index()

orders_products = pd.merge(orders_products, order_stat, on='order_id')

In [35]:
data1 = orders_products.groupby(['user_id', 'product_id']).agg({'user_id': 'size',
                                                               'order_number': ['min', 'max'],
                                                               'days_since_prior_order': ['mean', 'median'],
                                                               'order_dow': ['mean', 'median'],
                                                               'reordered': ['sum']})
data1.columns = data1.columns.droplevel(0)
data1.columns = [ 'up_first_order', 'up_last_order', 'order_dow_mean', 'order_dow_median','up_orders',
                'days_since_prior_order_mean', 'days_since_prior_order_median', 'reordered_sum']
data1['user_product_reordered_ratio'] = (data1.reordered_sum + 1.0) / data1.up_orders
data1.reset_index(inplace=True)

data1 = pd.merge(data1, prod_stat, on='product_id')
data1 = pd.merge(data1, user_stat, on='user_id')

data1['up_order_rate'] = data1.up_orders / data1.user_orders
data1['up_orders_since_last_order'] = data1.user_orders - data1.up_last_order
data1['up_order_rate_since_first_order'] = data1.user_orders / (data1.user_orders - data1.up_first_order + 1)

In [37]:
LP_check = data1[(data1['user_id'] == 1005126395)]
LP_check.to_csv('LP_check5.csv', index=False, header=True)

In [60]:
orders1 = orders.loc[orders.eval_set == 'prior', :]
orders_user = orders1[['order_id', 'user_id']]
labels = pd.merge(order_prior, orders_user, on=['user_id','order_id'])
labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()

In [70]:
train = pd.merge(labels, prod_usr, on='product_id')
train = pd.merge(train, prod_usr_reordered, on='product_id', how='left')
train.prod_users_unq_reordered.fillna(0, inplace=True)
train = pd.merge(train, data1, on=['product_id', 'user_id'])
train = pd.merge(train, aggregated, on=['user_id',  'product_id'])

In [71]:
LP_check = train[(train['user_id'] == 1005126395)]
LP_check.to_csv('LP_check6.csv', index=False, header=True)

In [72]:
dep = order_train.loc[:,['user_id','product_id','reordered']]

In [74]:
train = pd.merge(train, dep,on=['user_id',  'product_id'], how='left')
train.reordered.fillna(0, inplace=True)
train.fillna(999, inplace=True)
train = pd.merge(train, cust_map, on='user_id')

In [76]:
test = train[(train['rank_cust'] >= 900 )]
train = train[(train['rank_cust'] < 900 )]

In [77]:
train.shape

(83226, 35)

In [78]:
test.shape

(9331, 35)

In [81]:
LP_check = train[(train['user_id'] == 100012900)]
LP_check.to_csv('LP_check7.csv', index=False, header=True)

In [83]:
train.to_csv('train.csv', index=False, header=True)
test.to_csv('test.csv', index=False, header=True)

In [98]:
train_x = train.iloc[:,3:34]
test_x = test.iloc[:,3:33]

In [104]:
clf = gbm( loss = 'deviance' ,
          learning_rate= 0.01,n_estimators= 100, subsample= 0.75,verbose= 1)
clf.fit(train_x.drop('reordered', axis = 1) ,train_x['reordered'] )

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.5102           0.0021           24.97s
         2           0.5077           0.0019           19.38s
         3           0.5075           0.0019           16.72s
         4           0.5063           0.0017           15.79s
         5           0.5002           0.0017           14.82s
         6           0.4999           0.0016           14.19s
         7           0.5038           0.0015           13.63s
         8           0.4992           0.0014           13.24s
         9           0.4995           0.0014           12.83s
        10           0.4956           0.0014           12.51s
        20           0.4840           0.0010           10.82s
        30           0.4726           0.0008            9.30s
        40           0.4674           0.0006            7.89s
        50           0.4594           0.0005            6.50s
        60           0.4580           0.0004            5.25s
       

GradientBoostingClassifier(init=None, learning_rate=0.01, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              random_state=None, subsample=0.75, verbose=1,
              warm_start=False)

In [105]:
out = clf.predict(test_x) 
out = pd.DataFrame(out) 
#test_x = pd.concat((test,out), axis = 1)

In [ ]:
def ankur_search(prob, dtype=np.float32):
    size = len(prob)
    fk = np.zeros((size + 1), dtype=dtype)
    C = np.zeros((size + 1, size + 1), dtype=dtype)
    S = np.empty((2 * size + 1), dtype=dtype)
    S[:] = np.nan
    for k in range(1, 2 * size + 1):
        S[k] = 1./k
    roots = (prob - 1.0) / prob
    for k in range(size, 0, -1):
        poly = np.poly1d(roots[0:k], True)
        factor = np.multiply.reduce(prob[0:k])
        C[k, 0:k+1] = poly.coeffs[::-1]*factor
        for k1 in range(size + 1):
            fk[k] += (1. + 1.) * k1 * C[k, k1]*S[k + k1]
        for i in range(1, 2*(k-1)):
            S[i] = (1. - prob[k-1])*S[i] + prob[k-1]*S[i+1]

    return fk

none_product = 50000

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

def create_products(df):
    # print(df.product_id.values.shape)
    products = df.product_id.values
    prob = df.prediction.values

    sort_index = np.argsort(prob)[::-1]

    values = ankur_search(prob[sort_index][0:80], dtype=np.float64)

    index = np.argmax(values)

    print('iteration', df.shape[0], 'optimal value', index)

    best = ' '.join(map(lambda x: str(x) if x != none_product else 'None', products[sort_index][0:index]))
    df = df[0:1]
    df.loc[:, 'products'] = best
    return df



check  = data[(data['order_id'] == 65432) | (data['order_id'] == 203668)]
check  = data[data['order_id'] == 65432]


prob = check.prediction.values
sort_index = np.argsort(prob)[::-1]
values = fast_search(prob[sort_index][0:80], dtype=np.float64)
print(values)


gp = data.groupby('order_id')['not_a_product'].apply(lambda x: np.multiply.reduce(x.values)).reset_index()
gp.rename(columns={'not_a_product': 'prediction'}, inplace=True)
gp['product_id'] = none_product


data = applyParallel(data.groupby(data.order_id), create_products).reset_index()
